sheet flow:

- ~~define track of interest based on sheet_gid dict~~
- ~~read in weekend dataframe~~
- ~~run function drop_empties~~
- ~~run function check_df~~
- ~~produce these lists:~~
    - ~~predicted_qualifying_order, actual_qualifying_order, predicted_race_order, actual_race_order~~
- ~~produce these dfs:~~
    - ~~constructor_pricing, driver_pricing~~
- ~~produce these lists:~~
    - ~~drivers, constructors~~
- ~~set up constants for tracking weekend scores~~
    - ~~constructor_scores, driver_scores~~
- ~~run function score_qualification_order~~
- run function score_race_order
- run function score_fastest_pitstop
- run function score_fastest_lap
    
still to-do:
- ~~changing the overtake bonus/gain/loss points from estimations to direct reports in fantasy~~
- ~~produce df of driver and constructor pricing for making team affordability calculations each week, later when i get to the full program part of it~~
- ~~incorporate the team recommendation function that the post i found used~~
- ~~write function scoring the predictions only~~
    - ~~assume difference between predicted qualifying position and predicted race position are direct gain/loss~~
- ~~incorporate the pieces into scoring all of the race into one function~~
    - ~~qualifying, race, superlatives, overtakes+gain/loss~~
- make google sheet of teams and track team composition through each weekend
    - read this in, figure out if i can define a range of cells so i can do all three teams on the same google sheet or if i'm going to have to do a separate sheet for each
- ~~add scoring for my team to use for the team recommendations~~
- ~~add ability to handle DNF, DNQ, and OUT (sainz, stroll, zhou at saudi arabia)~~

In [1]:
from weekend_functions import *

### Define Track and Read In Weekend Data

In [2]:
track_name = 'china'

In [3]:
# read in from google sheets the sheet for this weekend's events
weekend_df = pd.read_csv(f'https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid={sheet_gid[track_name]}')

# read in from local sometimes needed when offline, make sure to download the file from sheets first...
# weekend_df = pd.read_csv(f'f1_main - {track_name}.csv')

weekend_df = drop_empties(weekend_df)
check_df(weekend_df)

weekend_df

,Team,Driver,predicted_qualifying_china,predicted_sprint_qualifying_china,actual_sprint_qualifying_china,predicted_sprint_race_china,predicted_race_china
0,Red Bull Racing-RBPT,Max Verstappen,1,4,4,1,1
1,Red Bull Racing-RBPT,Sergio Perez,4,6,6,3,2
2,Ferrari,Charles Leclerc,3,7,7,7,3
3,Ferrari,Carlos Sainz Jr.,2,5,5,4,4
4,Mercedes,Lewis Hamilton,8,2,2,6,6
5,McLaren-Mercedes,Lando Norris,5,1,1,2,7
6,Mercedes,George Russell,7,11,11,9,8
7,McLaren-Mercedes,Oscar Piastri,6,8,8,8,5
8,Aston Martin Aramco-Mercedes,Fernando Alonso,9,3,3,5,9
9,Aston Martin Aramco-Mercedes,Lance Stroll,11,15,15,16,10


In [4]:
weekend_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Team                               20 non-null     object
 1   Driver                             20 non-null     object
 2   predicted_qualifying_china         20 non-null     int64 
 3   predicted_sprint_qualifying_china  20 non-null     int64 
 4   actual_sprint_qualifying_china     20 non-null     int64 
 5   predicted_sprint_race_china        20 non-null     int64 
 6   predicted_race_china               20 non-null     int64 
dtypes: int64(5), object(2)
memory usage: 1.2+ KB


### Define Predicted Qualifying, Actual Qualifying, Predicted Race, Actual Race Orders, Driver Gain/Loss

this was used when writing functions to score the actual race. its not needed for what this sheet has become, picking a team for the next race. i'll move it to another sheet eventually when if/when i need it still. or i'll delete it.

In [5]:
# # qualifying orders
# predicted_qualifying_order = weekend_df[['Driver', f'predicted_qualifying_{track_name}']].sort_values(by=[f'predicted_qualifying_{track_name}'])['Driver'].tolist()
# actual_qualifying_order = weekend_df[['Driver', f'actual_qualifying_{track_name}']].sort_values(by=[f'actual_qualifying_{track_name}'])['Driver'].tolist()

# # race orders
# predicted_race_order = weekend_df[['Driver', f'predicted_race_{track_name}']].sort_values(by=[f'predicted_race_{track_name}'])['Driver'].tolist()
# if f'actual_race_{track_name}' in weekend_df.columns:
#     actual_race_order = weekend_df[['Driver', f'actual_race_{track_name}']].sort_values(by=[f'actual_race_{track_name}'])['Driver'].tolist()

# # driver_gain_loss
# driver_gain_loss_overtake = {}
# for driver in weekend_df.Driver:
#     driver_gain_loss_overtake[driver] = {}
#     driver_gain_loss_overtake[driver]['gain_loss'] = weekend_df.loc[weekend_df['Driver'] == driver][[f'positions_gained_{track_name}']].values[0][0]
#     driver_gain_loss_overtake[driver]['overtake'] = weekend_df.loc[weekend_df['Driver'] == driver][[f'overtake_bonus_{track_name}']].values[0][0]

### Driver and Constructor Prices

In [5]:
# pricing for drivers and constructors, updated each week with the latest values from the Formula 1 Fantasy Game
driver_pricing = pd.read_csv('https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid=920234107').dropna(axis=1, how='all')
constructor_pricing = pd.read_csv('https://docs.google.com/spreadsheets/d/14kBO9LAo4-uPrQlH6xm_Fm2OcNB15xUdnzUbIaRFjOU/export?format=csv&gid=1972297135').dropna(axis=1, how='all')

In [6]:
current_driver_values = {x[0]: x[1] for x in driver_pricing[['Driver', track_name]].values}
current_constructor_values = {x[0]: x[1] for x in constructor_pricing[['Constructor', track_name]].values}

### Look at Predictions for the Weekend

In [7]:
drivers, constructors, driver_scores, constructor_scores, driver_score_summary, constructor_score_summary = score_race_qualifying_sprint_predicted(weekend_df, track_name)

In [8]:
driver_scores

{'Max Verstappen': 64,
 'Sergio Perez': 41,
 'Charles Leclerc': 25,
 'Carlos Sainz Jr.': 26,
 'Lewis Hamilton': 14,
 'Lando Norris': 16,
 'George Russell': 11,
 'Oscar Piastri': 18,
 'Fernando Alonso': 6,
 'Lance Stroll': 2,
 'Yuki Tsunoda': 26,
 'Valtteri Bottas': 2,
 'Guanyu Zhou': -8,
 'Daniel Ricciardo': 6,
 'Alexander Albon': -4,
 'Pierre Gasly': -8,
 'Kevin Magnussen': 6,
 'Esteban Ocon': -6,
 'Nico Hulkenberg': 16,
 'Logan Sargeant': -1}

In [9]:
constructor_scores

{'Red Bull Racing-RBPT': 100,
 'Ferrari': 61,
 'Mercedes': 35,
 'McLaren-Mercedes': 44,
 'Aston Martin Aramco-Mercedes': 13,
 'VisaCashApp RB': 33,
 'Stake F1 Team Kick Sauber': -5,
 'Williams-Mercedes': 0,
 'Alpine-Renault': -11,
 'Haas-Ferrari': 21}

# Evaulate Next Race

In [15]:
# F-winning

my_constructors = ['Ferrari', 'Red Bull Racing-RBPT']
my_drivers = ['Sergio Perez', 'Nico Hulkenberg', 'Daniel Ricciardo', 'Pierre Gasly', 'Yuki Tsunoda']
remaining_cost_cap = 1.0

In [11]:
# my team predicted points and cost spent

team_score = sum(map(lambda x: driver_scores[x], my_drivers)) + max(map(lambda x: driver_scores[x], my_drivers)) + sum(map(lambda x: constructor_scores[x], my_constructors))
team_value = sum(map(lambda x: current_driver_values[x], my_drivers)) + sum(map(lambda x: current_constructor_values[x], my_constructors)) + remaining_cost_cap

print(f'Team score with {my_drivers} and {my_constructors}: {team_score}')
print(f'Team value as it is currently constructed: {round(team_value, 1)}')

Team score with ['Sergio Perez', 'Nico Hulkenberg', 'Daniel Ricciardo', 'Pierre Gasly', 'Yuki Tsunoda'] and ['Ferrari', 'Red Bull Racing-RBPT']: 283
Team value as it is currently constructed: 105.0


In [17]:
# potential_constructors = ['Ferrari', 'Red Bull Racing-RBPT']
# potential_drivers = ['Yuki Tsunoda', 'Charles Leclerc', 'Logan Sargeant', 'Alexander Albon', 'Lance Stroll']

# sum(map(lambda x: driver_scores[x], potential_drivers)) + max(map(lambda x: driver_scores[x], potential_drivers)) + sum(map(lambda x: constructor_scores[x], potential_constructors))

235

In [19]:
top_teams = main(my_drivers, my_constructors, weekend_df, track_name, driver_pricing, constructor_pricing, remaining_cost_cap)

=== Predicted Driver Scores for China ===
{'Max Verstappen': 64, 'Sergio Perez': 41, 'Charles Leclerc': 25, 'Carlos Sainz Jr.': 26, 'Lewis Hamilton': 14, 'Lando Norris': 16, 'George Russell': 11, 'Oscar Piastri': 18, 'Fernando Alonso': 6, 'Lance Stroll': 2, 'Yuki Tsunoda': 26, 'Valtteri Bottas': 2, 'Guanyu Zhou': -8, 'Daniel Ricciardo': 6, 'Alexander Albon': -4, 'Pierre Gasly': -8, 'Kevin Magnussen': 6, 'Esteban Ocon': -6, 'Nico Hulkenberg': 16, 'Logan Sargeant': -1}

=== Current Team ===
Constructors: ['Ferrari', 'Red Bull Racing-RBPT']
Drivers: ['Sergio Perez', 'Nico Hulkenberg', 'Daniel Ricciardo', 'Pierre Gasly', 'Yuki Tsunoda']
Current Team Value: 105.9
Current Available Value: 1.0
Total Number of Team Combinations: 697680
Total Number of Team Combinations I can afford: 456263
Explored all of the valid 456263 teams.

=== TEAM AT POSITION 1 WITH SCORE 305 ===
Constructor: ('Red Bull Racing-RBPT', 'VisaCashApp RB')
Drivers: ('Max Verstappen', 'Yuki Tsunoda', 'Daniel Ricciardo', 'Kev

### Sprint Dev

In [66]:
weekend_df.columns.to_list()

['Team',
 'Driver',
 'predicted_qualifying_china',
 'predicted_sprint_qualifying_china',
 'actual_sprint_qualifying_china',
 'predicted_sprint_china',
 'predicted_race_china']

In [68]:
weekend_df

,Team,Driver,predicted_qualifying_china,predicted_sprint_qualifying_china,actual_sprint_qualifying_china,predicted_sprint_china,predicted_race_china
0,Red Bull Racing-RBPT,Max Verstappen,1,4,4,1,1
1,Red Bull Racing-RBPT,Sergio Perez,4,6,6,3,2
2,Ferrari,Charles Leclerc,3,7,7,7,3
3,Ferrari,Carlos Sainz Jr.,2,5,5,4,4
4,Mercedes,Lewis Hamilton,8,2,2,6,6
5,McLaren-Mercedes,Lando Norris,5,1,1,2,7
6,Mercedes,George Russell,7,11,11,9,8
7,McLaren-Mercedes,Oscar Piastri,6,8,8,8,5
8,Aston Martin Aramco-Mercedes,Fernando Alonso,9,3,3,5,9
9,Aston Martin Aramco-Mercedes,Lance Stroll,11,15,15,16,10


In [97]:
[x for x in weekend_df.columns if 'predicted' in x]

['predicted_qualifying_japan', 'predicted_race_japan']

In [100]:
cols = ['Team', 'Driver']
cols.extend([x for x in weekend_df.columns if 'predicted' in x])

In [101]:
cols

['Team', 'Driver', 'predicted_qualifying_japan', 'predicted_race_japan']

In [102]:
predicted_df = weekend_df[cols]

In [103]:
predicted_df

,Team,Driver,predicted_qualifying_japan,predicted_race_japan
0,Red Bull Racing-RBPT,Max Verstappen,1,1
1,Ferrari,Charles Leclerc,2,3
2,Ferrari,Carlos Sainz Jr.,3,4
3,Red Bull Racing-RBPT,Sergio Perez,4,2
4,McLaren-Mercedes,Lando Norris,5,5
5,McLaren-Mercedes,Oscar Piastri,6,6
6,Mercedes,Lewis Hamilton,7,7
7,Mercedes,George Russell,8,8
8,Aston Martin Aramco-Mercedes,Fernando Alonso,9,9
9,Aston Martin Aramco-Mercedes,Lance Stroll,10,10


In [104]:
def score_race_qualifying_sprint_predicted(
    weekend_df,
    track_name):
    '''
    this function scores the predicted race and qualifying results in one go. only the predicted results.
    it takes the predicted from qualifying and the race, awards points for positions gained/lost and overtakes, and awards points for finish order in qualifying and in the race. 
    it awards fastest lap to the race winner because usually that's how it goes, but not always.
    
    parameters:
    track_name: str, track name as it appears in the sheet_gid dict for the purpose of loading the correct track.
    predicted_df: dataframe, a sub-df of the weekend_df that only has predicted qualifying and race positions for each driver, and non participations removed
    
    returns: 
    drivers: list
    constrctors: list
    driver_scores: dict, dict of the drivers' scores
    constructor_scores: dict, dict of the constructors' scores
    driver_score_summary: dict, breaking down the pieces of the drivers' scores
    constructor_score_summary: dict, breaking down the pieces of the constructors' scores
    '''
    
    # check if sprint race for scoring
    sprint_flag = False
    for col in weekend_df.columns:
        if 'sprint' in col:
            sprint_flag = True
    
    # some setup
    cols = ['Team', 'Driver']
    cols.extend([x for x in weekend_df.columns if 'predicted' in x])
    predicted_df = weekend_df[cols]
    driver_scores, constructor_scores, driver_score_summary, constructor_score_summary = {}, {}, {}, {}
    driver_to_constructor, constructor_to_driver = driver_constructor_mappings(predicted_df)
    predicted_qualifying_order = predicted_df.sort_values(by=[f'predicted_qualifying_{track_name}'])['Driver'].tolist()
    
    # Score race and qualifying order. Assume all drivers posted a qualifying time and finished the race.
    for driver in predicted_df.Driver:
        constructor = predicted_df.loc[predicted_df.Driver == driver]['Team'].values[0]
        driver_score_summary[driver] = {}
        driver_score_summary[driver]['constructor'] = constructor
        driver_score_summary[driver]['sprint_race'] = sprint_flag
        driver_points = 0
        constructor_points = 0
        
        # score quali and race positions, and determine gain/loss and overtakes
        quali_position = predicted_df.loc[predicted_df.Driver == driver][f'predicted_qualifying_{track_name}'].values[0]
        race_position = predicted_df.loc[predicted_df.Driver == driver][f'predicted_race_{track_name}'].values[0]

        # assume gain/loss and overtakes are strictly due to difference between qualifying and race positions
        # overtakes cannot be negative
        driver_gain_loss = quali_position - race_position
        driver_overtake = (driver_gain_loss if driver_gain_loss >= 0 else 0)
        
        driver_score_summary[driver]['quali_position'] = quali_position
        driver_score_summary[driver]['race_position'] = race_position
        driver_score_summary[driver]['gain_loss'] = driver_gain_loss
        driver_score_summary[driver]['overtake'] = driver_overtake
        
        # score driver and constructor for finishing position
        # +1-25 for position
        race_position_points = race_position_to_points.get(race_position)
        quali_position_points = quali_position_to_points.get(quali_position)
        
        # score sprint predictions
        sprint_position_points, driver_sprint_gain_loss, driver_sprint_overtake = 0, 0, 0
        
        if sprint_flag:
            sprint_quali_position = predicted_df.loc[predicted_df.Driver == driver][f'predicted_sprint_qualifying_{track_name}'].values[0]
            sprint_race_position = predicted_df.loc[predicted_df.Driver == driver][f'predicted_sprint_race_{track_name}'].values[0]
            
            driver_sprint_gain_loss = sprint_quali_position - sprint_race_position
            driver_sprint_overtake = (driver_sprint_gain_loss if driver_sprint_gain_loss >= 0 else 0)
            sprint_position_points = sprint_position_to_points.get(sprint_race_position)

            driver_score_summary[driver]['sprint_quali_position'] = sprint_quali_position
            driver_score_summary[driver]['sprint_race_position'] = sprint_race_position
            driver_score_summary[driver]['sprint_gain_loss'] = driver_sprint_gain_loss
            driver_score_summary[driver]['sprint_overtake'] = driver_sprint_overtake
            driver_score_summary[driver]['sprint_position_points'] = sprint_position_points
        
        # tally up the gain/loss, overtake, race position points, and quali position points
        driver_points += driver_gain_loss + driver_overtake + race_position_points + quali_position_points + sprint_position_points + driver_sprint_gain_loss + driver_sprint_overtake
        constructor_points += driver_gain_loss + driver_overtake + race_position_points + quali_position_points + sprint_position_points + driver_sprint_gain_loss + driver_sprint_overtake
        
        driver_score_summary[driver]['quali_position_points'] = quali_position_points
        driver_score_summary[driver]['race_position_points'] = race_position_points
        
        driver_scores = increase_score(driver_scores, driver, driver_points)
        constructor_scores = increase_score(constructor_scores, constructor, constructor_points)
    
        
    # Score constructors' qualification results based on how the drivers qualify. Assume all drivers post a qualifying position
    for constructor in predicted_df.Team.unique():
        constructor_score_summary[constructor] = {}
        
        # which qualifying position did each driver finish in
        team_qualifying_positions = predicted_df.loc[predicted_df.Team == constructor][f'predicted_qualifying_{track_name}'].tolist()
        
        # which qualifying round did each driver finish in
        team_qualifying_rounds = list(map(lambda p: quali_result(p), team_qualifying_positions))
        
        # points for qualifying positions
        quali_position_points = sum(map(lambda d: quali_position_to_points[d], team_qualifying_positions))

        q3_tot = [x for x in team_qualifying_rounds if x == 'Q3']
        q2_tot = [x for x in team_qualifying_rounds if x == 'Q2']

        # q3 finishers
        if len(q3_tot) == 2:
            team_quali_score = 10
        elif len(q3_tot) == 1:
            team_quali_score = 5
                
        # q2 finishers
        elif len(q2_tot) == 2:
            team_quali_score = 3
        elif len(q2_tot) == 1:
            team_quali_score = 1

        # nobody got past q1
        else:
            team_quali_score = -1
            
        constructor_scores = increase_score(constructor_scores, constructor, team_quali_score)
        
        constructor_score_summary[constructor]['quali_position'] = {q[0]: q[1] for q in predicted_df.loc[predicted_df.Team == constructor][['Driver', f'predicted_qualifying_{track_name}']].values}
        constructor_score_summary[constructor]['quali_position_points'] = quali_position_points
        constructor_score_summary[constructor]['quali_finish_points'] = team_quali_score
        
        
    # award fastest lap to driver predicted to finish race in P1
    fastest_driver = predicted_df.sort_values(f'predicted_race_{track_name}')['Driver'].tolist()[0]
    driver_scores = increase_score(driver_scores, fastest_driver, 10)

    return predicted_df.Driver.tolist(), predicted_df.Team.unique(), driver_scores, constructor_scores, driver_score_summary, constructor_score_summary

In [105]:
drivers, constructors, driver_scores, constructor_scores, driver_score_summary, constructor_score_summary = score_race_qualifying_sprint_predicted(weekend_df, track_name)

In [106]:
driver_scores

{'Max Verstappen': 45,
 'Charles Leclerc': 23,
 'Carlos Sainz Jr.': 19,
 'Sergio Perez': 29,
 'Lando Norris': 16,
 'Oscar Piastri': 13,
 'Lewis Hamilton': 10,
 'George Russell': 7,
 'Fernando Alonso': 4,
 'Lance Stroll': 2,
 'Yuki Tsunoda': 0,
 'Nico Hulkenberg': -1,
 'Kevin Magnussen': -1,
 'Daniel Ricciardo': 4,
 'Alexander Albon': 0,
 'Valtteri Bottas': 0,
 'Pierre Gasly': 0,
 'Logan Sargeant': 0,
 'Guanyu Zhou': 0,
 'Esteban Ocon': 0}

In [107]:
constructor_scores

{'Red Bull Racing-RBPT': 74,
 'Ferrari': 52,
 'McLaren-Mercedes': 39,
 'Mercedes': 27,
 'Aston Martin Aramco-Mercedes': 16,
 'VisaCashApp RB': 7,
 'Haas-Ferrari': 1,
 'Williams-Mercedes': 1,
 'Stake F1 Team Kick Sauber': -1,
 'Alpine-Renault': -1}

In [108]:
driver_score_summary['Max Verstappen']

{'constructor': 'Red Bull Racing-RBPT',
 'sprint_race': False,
 'quali_position': 1,
 'race_position': 1,
 'gain_loss': 0,
 'overtake': 0,
 'quali_position_points': 10,
 'race_position_points': 25}

In [109]:
constructor_score_summary['Red Bull Racing-RBPT']

{'quali_position': {'Max Verstappen': 1, 'Sergio Perez': 4},
 'quali_position_points': 17,
 'quali_finish_points': 10}